# Import required libraries

In [1]:
# pip install seaborn
# pip install xgboost
# pip install keras
# pip install sklearn
# pip install pandas
# pip install numpy

In [2]:
import pandas as pd

In [3]:
#  !pip install seaborn
#  !pip install xgboost
#  !pip install keras
#  !pip install sklearn
#  !pip install pandas
#  !pip install numpy


In [4]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

Using TensorFlow backend.


# Import the data

In [5]:
# help(pd.read_excel)

In [6]:
# df_x = pd.read_excel('Research data sample.xlsx', sheet_name = 'Sheet1')

In [7]:
df_x = pd.read_excel('Research data.xlsx', sheet_name = 'Sheet1')

In [8]:
df_x.head(2)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
df_x.columns

Index(['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6',
       'Column7', 'Column8', 'Column9', 'Column10', 'Column11', 'Column12',
       'Column13', 'Column14', 'Column15', 'Column16', 'Column17', 'Column18',
       'Column19', 'Column20', 'Column21'],
      dtype='object')

In [10]:
# df_y = pd.read_excel('Research data sample.xlsx', sheet_name = 'Sheet2')

In [11]:
df_y = pd.read_excel('Research data.xlsx', sheet_name = 'Sheet2')

In [12]:
df_y.head(2)

,Column1
0,66.193
1,66.193


In [13]:
df_y = df_y.rename(columns={'Column1': 'Position'})

In [14]:
df_y.head(2)

,Position
0,66.193
1,66.193


In [15]:
# df_all = df_x
# df_all['Position'] = df_y['Position'].values

In [16]:
# df_all.head(2)

In [ ]:
df_kor = df.corr()
plt.figure(figsize=(10,10))
sns.heatmap(df_kor, vmin=-1, vmax=1, cmap="viridis", annot=True, linewidth=0.1)

In [17]:
from sklearn.model_selection import train_test_split

train_x, test_x, train_y, test_y = train_test_split(df_x, df_y, test_size=0.2, random_state=42)

In [18]:
train_x.head(2)

,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column12,Column13,Column14,Column15,Column16,Column17,Column18,Column19,Column20,Column21
339610,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125963,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [19]:
train_x.shape[1]

21

# Trying a Deep Neural Network

#### Followed this blogpost: https://towardsdatascience.com/deep-neural-networks-for-regression-problems-81321897ca33

In [38]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(256, kernel_initializer='normal',input_dim = train_x.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(128, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(64, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(32, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 256)               5632      
_________________________________________________________________
dense_9 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 33        
Total params: 48,897
Trainable params: 48,897
Non-trainable params: 0
_________________________________________________________________


In [39]:
checkpoint_name = 'models/Weights-001--65.91199.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [40]:
callbacks_list

In [41]:
train = train_x.to_numpy()

In [42]:
train.shape

(583706, 21)

In [43]:
target = train_y.to_numpy()

In [44]:
target.shape

(583706, 1)

In [ ]:
NN_model.fit(train, target, epochs=500, batch_size=64, validation_split = 0.2, callbacks=callbacks_list)

Train on 466964 samples, validate on 116742 samples
Epoch 1/500
466964/466964 [==============================] - 24s 52us/step - loss: 67.0224 - mean_absolute_error: 67.0224 - val_loss: 65.9487 - val_mean_absolute_error: 65.9488

Epoch 00001: val_loss improved from inf to 65.94874, saving model to models/Weights-001--65.91199.hdf5
Epoch 2/500
466964/466964 [==============================] - 25s 53us/step - loss: 65.8887 - mean_absolute_error: 65.8887 - val_loss: 65.8441 - val_mean_absolute_error: 65.8441

Epoch 00002: val_loss improved from 65.94874 to 65.84411, saving model to models/Weights-001--65.91199.hdf5
Epoch 3/500
466964/466964 [==============================] - 26s 56us/step - loss: 65.8527 - mean_absolute_error: 65.8527 - val_loss: 65.8467 - val_mean_absolute_error: 65.8467

Epoch 00003: val_loss did not improve from 65.84411
Epoch 4/500
466964/466964 [==============================] - 25s 53us/step - loss: 65.7934 - mean_absolute_error: 65.7933 - val_loss: 66.4936 - val_mea

466964/466964 [==============================] - 18s 39us/step - loss: 65.5648 - mean_absolute_error: 65.5647 - val_loss: 65.7435 - val_mean_absolute_error: 65.7435

Epoch 00070: val_loss did not improve from 65.74221
Epoch 71/500
466964/466964 [==============================] - 18s 39us/step - loss: 65.5591 - mean_absolute_error: 65.5590 - val_loss: 65.7482 - val_mean_absolute_error: 65.7482

Epoch 00071: val_loss did not improve from 65.74221
Epoch 72/500
466964/466964 [==============================] - 19s 40us/step - loss: 65.5590 - mean_absolute_error: 65.5590 - val_loss: 65.7894 - val_mean_absolute_error: 65.7895

Epoch 00072: val_loss did not improve from 65.74221
Epoch 73/500
466964/466964 [==============================] - 18s 39us/step - loss: 65.5580 - mean_absolute_error: 65.5579 - val_loss: 65.7301 - val_mean_absolute_error: 65.7301

Epoch 00073: val_loss improved from 65.74221 to 65.73015, saving model to models/Weights-001--65.91199.hdf5
Epoch 74/500
466964/466964 [=====

466964/466964 [==============================] - 17s 37us/step - loss: 65.5294 - mean_absolute_error: 65.5294 - val_loss: 65.7551 - val_mean_absolute_error: 65.7551

Epoch 00140: val_loss did not improve from 65.73015
Epoch 141/500
466964/466964 [==============================] - 18s 39us/step - loss: 65.5368 - mean_absolute_error: 65.5369 - val_loss: 65.7408 - val_mean_absolute_error: 65.7409

Epoch 00141: val_loss did not improve from 65.73015
Epoch 142/500
466964/466964 [==============================] - 17s 37us/step - loss: 65.5308 - mean_absolute_error: 65.5307 - val_loss: 65.7422 - val_mean_absolute_error: 65.7422

Epoch 00142: val_loss did not improve from 65.73015
Epoch 143/500
466964/466964 [==============================] - 19s 41us/step - loss: 65.5323 - mean_absolute_error: 65.5322 - val_loss: 65.8108 - val_mean_absolute_error: 65.8108

Epoch 00143: val_loss did not improve from 65.73015
Epoch 144/500
466964/466964 [==============================] - 18s 38us/step - loss: 6

466964/466964 [==============================] - 18s 38us/step - loss: 65.5265 - mean_absolute_error: 65.5264 - val_loss: 65.7912 - val_mean_absolute_error: 65.7912

Epoch 00210: val_loss did not improve from 65.73015
Epoch 211/500
466964/466964 [==============================] - 18s 38us/step - loss: 65.5246 - mean_absolute_error: 65.5247 - val_loss: 65.7337 - val_mean_absolute_error: 65.7337

Epoch 00211: val_loss did not improve from 65.73015
Epoch 212/500
466964/466964 [==============================] - 18s 38us/step - loss: 65.5222 - mean_absolute_error: 65.5222 - val_loss: 65.7326 - val_mean_absolute_error: 65.7327

Epoch 00212: val_loss did not improve from 65.73015
Epoch 213/500
466964/466964 [==============================] - 18s 38us/step - loss: 65.5254 - mean_absolute_error: 65.5254 - val_loss: 65.7925 - val_mean_absolute_error: 65.7925

Epoch 00213: val_loss did not improve from 65.73015
Epoch 214/500
466964/466964 [==============================] - 19s 41us/step - loss: 6

In [28]:
wights_file = 'models/Weights-001--65.91199.hdf5' # choose the best checkpoint 
NN_model.load_weights(wights_file) # load it
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [29]:
test = test_x.to_numpy()

In [36]:
test_x.shape

(145927, 21)

In [30]:
predictions = NN_model.predict(test)

In [31]:
predictions[0]

array([120.623795], dtype=float32)

In [32]:
actual = test_y.to_numpy()

In [33]:
actual[0]

array([126.46])

### We will judge our model on Mean Absolute Error Metric

In [34]:
MAE = mean_absolute_error(actual , predictions)
print('Deep Neural Network validation MAE = ', MAE)

Deep Neural Network validation MAE =  65.8824274131749


# Trying CNN

In [35]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
# CNN Layers
model.add(Conv2D(32, (3, 3), input_shape=(300, 300, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# the model so far outputs 3D feature maps (height, width, features)

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))
# COMPILE
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])